# *Bounded Model Checking*

Nesta aula vamos estudar *model checking*, uma das técnicas mais utilizadas para verificação automática de propriedades de sistemas dinâmicos (sistemas caracterizados por um estado que evolui ao longo do tempo). O *model checking* tem por objectivo verificar automaticamente se uma propriedade (tipicamente especificada numa lógica temporal) é válida num modelo do sistema (tipicamente formalizado como um sistema de transição). Mais concretamente vamos estudar uma técnica de *bounded model checking*, onde o objectivo é verificar se uma propriedade é válida considerando apenas um dado número máximo de estados da execução do sistema.


Vamos usar a biblioteca [pySMT](https://github.com/pysmt/pysmt), apresentada na ficha 1, que permite que um programa em Python comunique com vários SMT solvers, tendo por base uma linguagem comum.
A documentação do pySMT pode ser encontrada em https://pysmt.readthedocs.io/en/latest/index.html.

## *First-order Transition Systems* (FOTS)

FOTS são modelos de sistemas dinâmicos que são determinados por um espaço de estados, uma relação de transição de estados e um conjunto de estados iniciais. Nos FOTS o conjunto de estados iniciais é descrito por um predicado unário (*init*) sobre o vector de variáveis associado ao espaço de estados, e a relação de transição é descrita por um predicado binário (*trans*) sobre dois vectores de variáveis que representam o estado antes e depois da transição.

### Modelação de programas com FOTS

Um programa pode ser modelado por um FOTS da seguinte forma:
- O estado é constituído pelas variáveis do programa mais uma variável para o respectivo *program counter*
- Os estados iniciais são caracterizados implicitamente por um predicado sobre as variáveis de estado
- As transições são caracterizadas implicitamente por um predicado sobre pares de estados

Considere, por exemplo o programa seguinte, anotado com uma pré-condição que restringe o valor inicial de $x$:

```Python
{ x >= 3 }
0: while (x>0):
1:    x = x-1
2: stop
```

Neste caso o estado do FOTS respectivo será um par de inteiros, o primeiro contendo o valor do $\mathit{pc}$ (o *program counter* que neste caso pode ser 0, 1 ou 2) e o segundo o valor da variável $x$. O estado inicial é caracterizado pelo seguinte predicado:

$$
\mathit{pc} = 0 \wedge x \ge 3
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge x > 0 \wedge \mathit{pc}' = 1 \wedge x' = x)\\
\vee\\
(\mathit{pc} = 0 \wedge x \le 0 \wedge \mathit{pc}' = 2 \wedge x' = x) \\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 0 \wedge x' = x - 1) \\
\vee\\
(\mathit{pc} = 2 \wedge \mathit{pc}' = 2 \wedge x' = x)
\end{array}
$$

Note que este predicado é uma disjunção de todas as possíveis transições que podem ocorrer no programa. Cada transição é caracterizada por um predicado onde uma variável do programa denota o seu valor no pré-estado e a mesma variável com apóstrofe denota o seu valor no pós-estado. É usual exigir que cada estado do FOTS tenha pelo menos um sucessor, pelo que o estado final do programa é caracterizado por uma transição para ele próprio (um lacete).

Usando estes predicados podemos usar um SMT solver para, por exemplo, gerar uma possível execução de $k\!-\!1$ passos do programa (em que $k>0$). Para tal precisamos de criar $k$ cópias das variáveis que caracterizam o estado do FOTS e depois impor que a primeira cópia satisfaz o predicado inicial e que cada par de cópias consecutivas satisfazem o predicado de transição.

Começamos por importar o módulo `pysmt.shortcuts` que oferece uma API simplificada que disponibiliza as funcionalidades para a utilização usual de um SMT solver.
Os tipos estão definidos no módulo `pysmt.typing` de onde temos que importar o tipo `INT`.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

A seguinte função cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),INT)
    return state

### Exercício 1

Defina a função `init` que, dado um possível estado do programa (um dicionário de variáveis), devolva um predicado do pySMT que testa se esse estado é um possível estado inicial do programa.

In [3]:
def init(state):
    # completar
    return And(state['pc'] == Int(0), state['x'] >= Int(3))

### Exercício 2

Defina a função `trans` que, dados dois possíveis estados do programa, devolva um predicado do pySMT que testa se é possível transitar do primeiro para o segundo.

In [4]:
def trans(curr,prox):
    """
    Defines a transition function for a state machine.

    Args:
        curr (dict): A dictionary representing the current state with keys 'pc' and 'x'.
        prox (dict): A dictionary representing the next state with keys 'pc' and 'x'.

    Returns:
        z3.ExprRef: A Z3 expression representing the transition conditions between the current and next states.
    """
    def trans(curr, prox):
        return Or(
            And(curr['pc'] == Int(0), curr['x'] > Int(0), prox['pc'] == Int(1), prox['x'] == curr['x']),
            And(curr['pc'] == Int(0), curr['x'] <= Int(0), prox['pc'] == Int(2), prox['x'] == curr['x']),
            And(curr['pc'] == Int(1), prox['pc'] == Int(0), prox['x'] == curr['x'] - Int(1)),
            And(curr['pc'] == Int(2), prox['pc'] == Int(2), prox['x'] == curr['x'])
        )

### Exercício 3

Complete a função de ordem superior `gera_traco` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e um número positivo `k`, use o SMT solver para gerar um possível traço de execução do programa de tamanho `k`. Para cada estado do traço deverá imprimir o respectivo valor das variáveis.

In [7]:
from z3 import *

def gera_traco(declare, init, trans, k):
    solver = Solver()
    
    # Declarar uma lista para armazenar os estados
    estados = [declare(i) for i in range(k)]
    
    # Adicionar a restrição que o primeiro estado deve ser o inicial
    solver.add(init(estados[0]))
    
    # Adicionar as transições entre os estados
    for i in range(k - 1):
        solver.add(trans(estados[i], estados[i + 1]))
    
    # Tentar encontrar um modelo que satisfaça as restrições
    if solver.check() == sat:
        modelo = solver.model()
        
        # Para cada estado, imprimir as variáveis
        for i in range(k):
            print(f"Estado {i}:")
            estado_atual = estados[i]
            for var in estado_atual:
                print(f"  {var} = {modelo[var]}")
    else:
        print("Nenhuma solução encontrada.")

# Exemplo de como as funções `declare`, `init` e `trans` poderiam ser implementadas:

def declare(i):
    # Declarar a variável do estado (no Z3, as variáveis são objetos)
    x = Int(f'x_{i}')
    return [x]

def init(estado):
    # Definir que o estado inicial tem x = 0
    return estado[0] == 0

def trans(estado1, estado2):
    # Definir a transição: x[i+1] = x[i] + 1
    return estado2[0] == estado1[0] + 1

# Exemplo de uso:
gera_traco(declare, init, trans, 10)


Estado 0:
  x_0 = 0
Estado 1:
  x_1 = 1
Estado 2:
  x_2 = 2
Estado 3:
  x_3 = 3
Estado 4:
  x_4 = 4
Estado 5:
  x_5 = 5
Estado 6:
  x_6 = 6
Estado 7:
  x_7 = 7
Estado 8:
  x_8 = 8
Estado 9:
  x_9 = 9


## Lógica temporal linear (LTL)

Sobre este FOTS podemos querer verificar várias propriedades temporais, como por exemplo:
1. $x$ é sempre não negativo
2. $x$ é sempre diferente de 1
3. $x$ chega inevitavelmente a 0
4. $x$ chega inevitavelmente a um valor negativo
5. o programa termina

Estas propriedades são de natureza muito diferente. As duas primeiras são *propriedades de segurança* (*safety*) que, em geral, garantem que "nada de mau irá acontecer". Neste caso em particular são invariantes, ou seja, propriedades que devem ser verdade em todos os estados da execução do programa. As três últimas são *propriedades de animação* (*liveness*) que, em geral, garantem que "algo de bom irá acontecer".

A lógica LTL introduz *operadores temporais* que nos permitem escrever estas propriedades formalmente. Os operadores mais conhecidos são o $G$, que informalmente significa "*é sempre verdade que*", e o $F$, que informalmente significa "*é inevitável que*". Com estes operadores, as propriedades acima podem ser especificadas formalmente do seguinte modo
1. $G\ (x \ge 0)$
2. $G\ (x \neq 1)$
3. $F\ (x = 0)$
4. $F\ (x < 0)$
5. $F\ (pc = 2)$

## *Bounded Model Checking* (BMC) para LTL

Como é óbvio, nem todas estas propriedades são válidas. Em particular a 2ª e a 4ª não o são. O objectivo da verificação é precisamente determinar se uma propriedade temporal é válida num FOTS ou não. Este procedimento designa-se *model checking* e, quando uma propriedade não é válida, produz um contra-exemplo (um traço do FOTS correspondente a uma execução do programa onde a propriedade falha). Nesta aula vamos estudar uma técnica particular de *model checking* designada *bounded model checking*, onde o objectivo é determinar se uma propriedade temporal é válida nos primeiros $K$ estados da execução do FOTS.

A ideia passa por usar um SMT solver para tentar descobrir um contra-exemplo para uma dada propriedade. Para simplificar vamos abordar apenas dois casos: verificação de invariantes da forma $G\ \phi$ e propriedades de animação simples da forma $F\ \phi$, em que $\phi$ é uma fórmula sem operadores temporais.

### BMC de invariantes

Para fazer BMC de um invariante, por exemplo $G\ (x \ge 0)$ basta-nos usar o solver para encontrar um contra-exemplo com no máximo $K$ estados onde a propriedade $(x \ge 0)$ seja inválida nalgum estado. Para tal podemos implementar um procedimento iterativo que tenta encontrar esse contra-exemplo com tamanhos crescentes, começando com tamanho 1 até ao tamanho máximo $K$. Para cada tamanho $0 < k \le K$ basta tentar encontrar um possível traço onde o invariante a verificar seja inválido no último estado. Para tal podemos usar um código muito semelhante ao da função `gera_traco`. O procedimento é interrompido mal um contra-exemplo seja encontrado, sendo garantido que esse é um contra-exemplo mínimo para essa propriedade.

### Exercício 4

Complete a definição da função de ordem superior `bmc_always` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, um invariante a verificar, e um número positivo `K`, use o SMT solver para verificar se esse invariante é sempre válido nos primeiros `K-1` passos de execução do programa, ou devolva um contra-exemplo mínimo caso não seja.

In [22]:
def bmc_always(declare,init,trans,inv,K):
    # completar

def nonnegative(state):
    return GE(state['x'], Int(0))

bmc_always(declare,init,trans,nonnegative,20)

> Invariant holds for the first 20 states.


### Exercício 5

Use o procedimento `bmc_always` para encontrar um contra-exemplo para a segunda propriedade especificada acima.

In [23]:
# completar

bmc_always(declare,init,trans,notone,20)

> Invariant does not hold for 5 first states. Counter-example:
> State 0: x = 3, pc= 0.
> State 1: x = 3, pc= 1.
> State 2: x = 2, pc= 0.
> State 3: x = 2, pc= 1.
> State 4: x = 1, pc= 0.


### BMC de propriedades de animação

Para fazer BMC de propriedades de animação da forma $F\ \phi$ o prodimento é ligeiramente mais complicado. Neste caso não basta encontrar um traço aberto com no máximo $K$ estados onde $\phi$ nunca seja válida, pois tal contra-exemplo não seria convincente: nada garante que $\phi$ não pudesse ser válida num ponto mais tarde da execução. Neste caso, o contra-exemplo teria que ser uma execução completa do programa que demonstre inequivocamente que a propriedade não é válida. É possível encontrar uma execução completa do programa com no máximo $K$ estados se a mesma tiver um *loop* no último estado, mais concretamente, uma transição para um dos estados precedentes.

### Exercício 6

Complete a definição da função de ordem superior `bmc_eventually` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, uma propriedade cuja inevitabilidade se pretende verificar, e um número positivo `K`, use o SMT solver para encontrar um contra-exemplo para essa propriedade considerando apenas os primeiros `K` estados de execução do programa. Note que neste caso um contra-exemplo tem que ser necessariamente um *loop* (no sentido acima referido) onde a propriedade desejada nunca seja válida.

In [32]:
def bmc_eventually(declare,init,trans,prop,bound):
    # completar
    
def zero(state):
    return Equals(state['x'],Int(0))

bmc_eventually(declare,init,trans,zero,20)

def terminates(state):
    return Equals(state['pc'],Int(2))

bmc_eventually(declare,init,trans,terminates,20)

> Property holds for execution of length 20.
> Property holds for execution of length 20.


### Exercício 7

Modifique a função `bmc_eventually` para não só imprimir os estados do contra-exemplo, mas também o estado onde começa o *loop*. Sugere-se a utilização do método `get_py_value` do pySMT para detectar esse estado. Este método permite avaliar uma expressão no modelo devolvendo o seu valor num tipo do Python.

Utilize esta função modificada para encontrar um contra exemplo para a quarta propriedade acima referida.

In [ ]:
def bmc_eventually(declare,init,trans,prop,bound):
    # completar

def negative(state):
    return (state['x'] < 0)

bmc_eventually(declare,init,trans,negative,10)

# Verificação Indutiva

Até agora vimos como modelar sistemas dinâmicos com *First-order Transition Systems* (FOTSs) e como verificar propriedades de segurança e animação sobre estes sistemas com *Bounded Model Checking* (BMC). O procedimento de BMC verifica se uma propriedade é válida num FOTS considerando apenas um número limitado de estados de execução. Se quisermos verificar uma propriedade para qualquer execução não limitada do FOTS temos que usar um procedimento alternativo. Vamos agora usar indução para fazer verificação em traços não limitadados.

## Verificação indutiva de invariantes

No caso da verificação de propriedades de segurança $G\ \phi$, para verificar o invariante $\phi$ por indução temos que verificar as seguintes condições:
- $\phi$ é válido nos estados iniciais, ou seja, $\mathit{init}(s) \rightarrow \phi(s)$
- Para qualquer estado, assumindo que $\phi$ é verdade, se executarmos uma transição, $\phi$ continua a ser verdade no próximo estado, ou seja, $\phi(s) \wedge \mathit{trans}(s,s') \rightarrow \phi(s')$.

### Exercício 8

Implemente a função `induction_always` para verificação de invariantes por indução. 
A função recebe como argumento uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e o invariante.

Note que terá que testar a validade das duas condições acima recorrendo à satisfiabilidade, ou seja, usando o solver para encontrar contra-exemplos, devendo o procedimento reportar qual das propriedades falha. Por exemplo, no caso da primeira deve procurar uma valoração que satisfaça $\mathit{init}(s) \wedge \neg \phi(s)$.

In [ ]:
def induction_always(declare,init,trans,inv):
    # completar

Usando este procedimento podemos tentar verificar a propriedade $G (x \ge 0)$ sobre o nosso programa.

In [ ]:
induction_always(declare,init,trans,nonnegative)

Como se pode verificar, a propriedade anterior, embora seja verdade, não pode ser verificada por indução, pois o passo indutivo falha quando $\mathit{pc} = 1$ e $x = 0$. De facto, este contra-exemplo para o passo de indução não é realista, dado que este estado não é acessível a partir do estado inicial. Para verificarmos esta propriedade por indução temos que encontrar um invariante $\phi$ que seja mais forte que o desejado, ou seja, $\phi(x,\mathit{pc}) \rightarrow x \ge 0$, e que seja verificável por indução.

### Exercício 9

Encontre uma propriedade mais forte que $x \ge 0$ e que seja verificável por indução. Sugestão: pode ser útil usar a funcção `Ite` do pySMT, que corresponde a um if-then-else lógico.

In [ ]:
# completar

def stronger_prop(s):
    inside_loop = Equals(s['pc'],Int(1))
    return Ite(inside_loop,GT(s('x')))


induction_always(declare,init,trans,stronger_prop)

### Exercício 10

Utilize o SMT solver para verificar que essa propriedade é realmente mais forte que $x \ge 0$.

In [9]:
# completar

def is_stronger(p2,p1):
    s = declare(0)
    with Solver() as solver:
        # para p1 nao ser tao ou mais forte , entao p1 ^ Not(p2) tem de ser SAT
        solver.push()
        solver.add_assertion(And(p1(s)),Not(p2(s)))
        if solver.solve():
            print("> p1 nao implica p2")
            flag = True
        else:
            print(">p1 nao implica p2. p2 nao é mais forte")
            flag = False
            solver.pop()
            
            
        solver.add_assertion(And(p2(s)),Not(p1(s))) 
        if solver.solve():
            print("> p2 nao implica p1")
            flag = True
        else:
            print(">p1 nao implica p2. p2 nao é mais forte")
            is flag:
                print(">p2 nao e mais forte")
 
 
 is_stronger(stronger_prop,nonnegative)       
        #Para p2 ser mais forte, p2 ^ Not(p1) is not SAT

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 28)

Nem sempre é fácil encontrar um invariante indutivo mais forte que o desejado. Um procedimento de verificação alternativo que pode ser utilizado nesses casos é a designada $k$-indução. A ideia é generalizar a indução simples assumindo no passo indutivo que o invariante é válido nos $k$ estados anteriores. Claro que neste caso também será necessário verificar que a propriedade é válida nos $k$ primeiros estados. Com esta generalização é possível verificar que $x \ge 0$ é um invariante recorrendo a $k$-indução com $k=2$.

### Exercício 11

Implemente o método `kinduction_always` para verificação de invariantes por $k$-indução.

In [ ]:
def kinduction_always(declare,init,trans,inv,k):
    # completar

## Verificação de propriedades de animação por indução

Como vimos, podemos verificar propriedades de animação do tipo $F\ \phi$ usando BMC. Mais uma vez, se quisermos verificar estas propriedades para qualquer execução ilimitada temos que usar um procedimento alternativo. Uma possibilidade consiste em reduzir a verificação dessas propriedades à verificação de uma propriedade de segurança, mais concretamente um invariante, que possa ser verificado por indução.

Por exemplo, uma técnica que pode ser usada para verificar uma propriedade de animação do tipo $F\ (G\ \phi)$, passa por descobrir um *variante* $V$ que satisfaz as seguintes condições:
- O variante nunca é negativo, ou seja, $G\ (V(s) \ge 0)$
- O variante descresce sempre (estritamente) ou atinge o valor 0, ou seja, $G\ (\forall s' . \mathit{trans}(s,s') \rightarrow (V(s') < V(s) \vee V(s') = 0))$
- Quando o variante é 0 verifica-se necessariamente $\phi$, ou seja, $G\ (V(s)=0 \rightarrow \phi(s))$

A terminação de um programa é um dos exemplos de propriedade que encaixa neste padrão. Por exemplo, para o programa acima a terminação pode ser expressa por $F\ (G\ \mathit{pc}=2)$ (note que devido ao lacete no estado final, o programa permanece indefinidamente no estado final).

Note que a segunda condição recorre a uma quantificação universal sobre os possíveis estados alcançáveis por uma transição a partir de cada estado.

### Exercício 12

Descubra o variante que permite provar por indução que o programa acima termina. Encontre também os menores valores de $k$ que permitem provar as condições acima por $k$-indução.

In [ ]:
# completar

### Exercício 13

Encontrar um variante que decresça em todas as transições nem sempre é fácil. Podemos relaxar a segunda condição acima e exigir que o variante apenas tenha que decrescer estritamente a cada $l$ transições. Neste caso dizemos que temos um *lookahead* de $l$. Altere a definição da função decrescente por forma a considerar um lookahead de 2 e encontre um variante mais simples que o anterior que lhe permita verificar a terminação do programa.

In [ ]:
# completar